In [8]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer

from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier

from bayes_opt import BayesianOptimization

from sklearn.metrics import f1_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

In [9]:
train = pd.read_csv('train_dataset.csv', encoding='cp949')

# 필요없는 열 제거
drop_cols = ["Unnamed: 0", "url_chinese_present", "html_num_tags('applet')"]
train.drop(drop_cols, axis=1, inplace=True)

# benign(정상) -> 1, malicious(악성) -> -1 
train['Result_v1'] = train['Result_v1'].map({'benign': 1, 'malicious': -1})

## <span style="color: hotpink"> **1. 데이터 전처리** </span>

### <span style="color: skyblue"> **결측치 처리(KNN Imputer)** </span>

In [10]:
imputer = KNNImputer(n_neighbors=9)

imputer.fit(train)
x = imputer.transform(train)

train = pd.DataFrame(x, columns=train.columns, index=train.index)

### <span style="color: skyblue"> **학습 데이터 분할** </span>

In [11]:
# Target 설정
target = 'Result_v1'

# 데이터 분리
X = train.drop(target, axis=1)
y = train.loc[:, target]

# 학습 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=2022)

## <span style="color: hotpink"> **2. 모델 하이퍼파라미터 튜닝(Bayesian optimization)** </span>

### <span style="color: skyblue"> **탐색 대상 함수 정의** </span>
- Catboost
- RandomForest
- XGboost
- LGBM
- Extratrees

In [12]:
# Catboost
def catboost_cv(subsample, depth, learning_rate, bagging_temperature, l2_leaf_reg):

    model = CatBoostClassifier(subsample=(subsample),
                               depth=int(depth),
                               learning_rate=(learning_rate),
                               bagging_temperature=(bagging_temperature),
                               l2_leaf_reg=(l2_leaf_reg),
                               random_state=2022,
                               silent=True
                               )

    model.fit(X_train, y_train)
    y_pred= model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred)

    return f1

# Randomforest
def rf_cv(n_estimators, max_depth, min_samples_split, min_samples_leaf):

    model = RandomForestClassifier(n_estimators=int(n_estimators),
                                   max_depth=int(max_depth),
                                   min_samples_split=int(min_samples_split),
                                   min_samples_leaf=int(min_samples_leaf),
                                   random_state=2022
                                  )

    model.fit(X_train, y_train)
    y_pred= model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred)

    return f1

# Xgboost
def xg_cv(learning_rate, max_depth, n_estimators, subsample):

    model = XGBClassifier(learning_rate=(learning_rate),
                          max_depth=int(max_depth),
                          n_estimators=int(n_estimators),
                          subsample=(subsample),
                          random_state=2022
                         )
                                      
    y_train_xg, y_valid_xg = y_train.copy(), y_valid.copy()
    y_train_xg = y_train_xg.map({1:1, -1:0})
    y_valid_xg = y_valid_xg.map({1:1, -1:0})                         

    model.fit(X_train, y_train_xg)
    y_pred= model.predict(X_valid)
    f1 = f1_score(y_valid_xg, y_pred)

    return f1

# LightGBM
def lgbm_cv(learning_rate, max_depth, n_estimators, num_leaves, subsample):

    model = LGBMClassifier(learning_rate=(learning_rate),
                            max_depth=int(max_depth),
                            n_estimators=int(n_estimators),
                            num_leaves=int(num_leaves),
                            subsample=(subsample),
                            random_state=2022
                                  )

    model.fit(X_train, y_train)
    y_pred= model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred)

    return f1

# ExtraTrees 
def xtree_cv(n_estimators, max_depth, min_samples_split, min_samples_leaf,):

    model = ExtraTreesClassifier(n_estimators=int(n_estimators),
                                 max_depth=int(max_depth),
                                 min_samples_split=int(min_samples_split),
                                 min_samples_leaf=int(min_samples_leaf),
                                )

    model.fit(X_train, y_train)
    y_pred= model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred)

    return f1

### <span style="color: skyblue"> **Catboost 하이퍼파라미터 튜닝** </span>

In [13]:
pds = {'subsample': (0.01, 1.0),
       'depth': (4, 16),
       'learning_rate': (0.001, 1.0),
       'bagging_temperature': (1.0, 10.0),
       'l2_leaf_reg': (1, 10)
      }

optimizer=BayesianOptimization(catboost_cv, pds, verbose=1, random_state=2022)    

optimizer.maximize(init_points=5,   
                   n_iter=100, 
                   acq='ei', 
                   xi=0.01)

print(optimizer.max)

|   iter    |  target   | baggin... |   depth   | l2_lea... | learni... | subsample |
-------------------------------------------------------------------------------------
|  3        |  0.9723   |  1.185    |  15.64    |  8.492    |  0.2131   |  0.19     |
|  6        |  0.9723   |  2.181    |  14.59    |  8.457    |  0.6524   |  0.4638   |
|  11       |  0.9736   |  5.639    |  11.71    |  9.958    |  0.09335  |  0.7156   |
|  16       |  0.9737   |  4.141    |  13.7     |  9.768    |  0.0253   |  0.8288   |
|  32       |  0.975    |  7.367    |  13.3     |  3.97     |  0.06901  |  0.924    |
|  104      |  0.9763   |  8.675    |  15.32    |  8.436    |  0.8024   |  0.2465   |
{'target': 0.9763157894736844, 'params': {'bagging_temperature': 8.67523211231834, 'depth': 15.324250797621403, 'l2_leaf_reg': 8.435591915591619, 'learning_rate': 0.8023802556747033, 'subsample': 0.2464941405066032}}


### <span style="color: skyblue"> **RandomForest 하이퍼파라미터 튜닝** </span>

In [23]:
pds = {'n_estimators': (10, 2000),
       'max_depth': (1, 300),
       'min_samples_split': (2, 100),
       'min_samples_leaf': (1, 100)
      }

optimizer=BayesianOptimization(rf_cv, pds, verbose=1, random_state=2022)    

optimizer.maximize(init_points=10,  
                   n_iter=500, 
                   acq='ei')   

print(optimizer.max)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  2        |  0.918    |  205.9    |  49.21    |  89.97    |  1.298e+0 |
|  6        |  0.9349   |  223.5    |  29.96    |  31.27    |  1.507e+0 |
|  9        |  0.9429   |  292.8    |  4.744    |  79.84    |  722.2    |
|  12       |  0.9468   |  291.0    |  2.6      |  53.29    |  748.0    |
|  13       |  0.9562   |  298.0    |  4.261    |  3.721    |  726.5    |
|  17       |  0.9588   |  275.6    |  3.657    |  8.383    |  685.5    |
|  20       |  0.9654   |  254.0    |  1.651    |  6.338    |  560.3    |
|  87       |  0.9655   |  218.5    |  2.901    |  3.808    |  231.9    |
|  158      |  0.9657   |  172.3    |  1.292    |  4.444    |  57.7     |
|  424      |  0.9668   |  15.43    |  1.212    |  3.269    |  1.38e+03 |
{'target': 0.9667994687915006, 'params': {'max_depth': 15.428571037861042, 'min_samples_leaf': 1.212272381999287

### <span style="color: skyblue"> **XGboost 하이퍼파라미터 튜닝** </span>

In [24]:
pds = {'learning_rate': (0.01, 1.0),
       'max_depth': (1, 300),
       'n_estimators': (10, 2000),
       'subsample': (0.1, 1.0)
      }

optimizer=BayesianOptimization(xg_cv, pds, verbose=1, random_state=2022)    

optimizer.maximize(init_points=10,  
                   n_iter=300, 
                   acq='ei')   

print(optimizer.max)

|   iter    |  target   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------
|  2        |  0.9643   |  0.6886   |  146.6    |  1.796e+0 |  0.6827   |
|  3        |  0.9658   |  0.898    |  216.6    |  1.664e+0 |  0.8448   |
|  7        |  0.9683   |  0.02848  |  157.6    |  1.73e+03 |  0.45     |
|  13       |  0.9723   |  0.01079  |  122.4    |  1.739e+0 |  0.698    |
|  95       |  0.9735   |  0.03136  |  109.3    |  675.6    |  0.8344   |
|  123      |  0.9736   |  0.1319   |  191.4    |  1.277e+0 |  0.562    |
|  178      |  0.975    |  0.4811   |  213.4    |  903.9    |  0.952    |
{'target': 0.9749670619235837, 'params': {'learning_rate': 0.4811138987024146, 'max_depth': 213.40262449461045, 'n_estimators': 903.9204388874033, 'subsample': 0.9520311177576255}}


### <span style="color: skyblue"> **LGBM 하이퍼파라미터 튜닝** </span>

In [25]:
pds = {'learning_rate': (0.01, 1.0),
       'max_depth': (1, 300),
       'n_estimators': (10, 2000),
       'num_leaves': (2, 600),
       'subsample': (0.01, 1.0)
      }

optimizer=BayesianOptimization(lgbm_cv, pds, verbose=1, random_state=2022)    

optimizer.maximize(init_points=10,  
                   n_iter=300, 
                   acq='ei')   

print(optimizer.max)

|   iter    |  target   | learni... | max_depth | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------
|  2        |  0.9696   |  0.4921   |  269.4    |  1.298e+0 |  538.4    |  0.7239   |
|  4        |  0.9697   |  0.4999   |  102.5    |  1.243e+0 |  586.6    |  0.1055   |
|  5        |  0.9723   |  0.7468   |  88.46    |  604.4    |  452.0    |  0.02848  |
|  9        |  0.9735   |  0.09057  |  13.69    |  600.6    |  219.5    |  0.4954   |
|  26       |  0.9736   |  0.2574   |  80.89    |  1.833e+0 |  369.7    |  0.5243   |
|  119      |  0.9737   |  0.454    |  163.3    |  796.7    |  360.5    |  0.8105   |
|  140      |  0.9763   |  0.7013   |  232.8    |  1.631e+0 |  356.9    |  0.4237   |
{'target': 0.9763157894736844, 'params': {'learning_rate': 0.7013120149929594, 'max_depth': 232.76242715728054, 'n_estimators': 1631.052868388154, 'num_leaves': 356.935675558698, 'subsample': 0.4236923048657558}}


### <span style="color: skyblue"> **ExtraTrees 하이퍼파라미터 튜닝** </span>

In [26]:
pds = {'n_estimators': (10, 2000),
       'max_depth': (1, 300),
       'min_samples_split': (2, 100),
       'min_samples_leaf': (1, 100)
      }

optimizer=BayesianOptimization(xtree_cv, pds, verbose=1, random_state=2022)    

optimizer.maximize(init_points=10,  
                   n_iter=300, 
                   acq='ei')   

print(optimizer.max)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  2        |  0.8235   |  205.9    |  49.21    |  89.97    |  1.298e+0 |
|  6        |  0.8241   |  223.5    |  29.96    |  31.27    |  1.507e+0 |
|  9        |  0.8886   |  292.8    |  4.744    |  79.84    |  722.2    |
|  11       |  0.9195   |  296.2    |  2.823    |  86.48    |  680.8    |
|  14       |  0.9253   |  279.9    |  1.914    |  74.01    |  690.2    |
|  33       |  0.9319   |  266.6    |  2.576    |  42.04    |  711.8    |
|  34       |  0.937    |  252.3    |  1.919    |  32.72    |  710.9    |
|  55       |  0.9435   |  284.2    |  1.867    |  18.1     |  735.4    |
|  67       |  0.9529   |  232.6    |  1.17     |  9.229    |  653.0    |
|  82       |  0.9554   |  237.0    |  1.04     |  6.038    |  693.4    |
|  87       |  0.9592   |  244.1    |  1.79     |  2.572    |  655.6    |
|  186      |  0.9606   |  252.0    | 